# **Task : 대규모 도로명 주소 데이터에서 동대문구 행정동 추출하기**
1) 카카오 앱 화면에서 프로젝트명 등록 (https://developers.kakao.com/console/app)
2) 앱 > '플랫폼 키' 선택하여 REST API Key 발급
3) API를 사용하여 도로명 주소 -> 좌표(위도, 경도) -> 행정동 변환

--

In [1]:
pip install PyKakao --upgrade

In [4]:
from PyKakao import Local
api = Local(service_key = "???") #본인 api 키 입력

In [6]:
import pandas as pd
hos = pd.read_excel('의료기관 목록.xlsx')

#세부 주소는 삭제하고 도로명주소만 전처리
hos["전처리_도로명주소"] = (
    hos["의료기관주소(도로명)"]
    .astype(str)
    .str.split(",")
    .str[0]
    .str.strip()
)

hos.head()

,순번,의료기관명,의료기관종별,설립구분,의료기관우편번호(도로명),의료기관주소(도로명),의료기관전화번호,의료기관팩스번호
0,1,솔라시도한방병원,한방병원,개인,'02561,"서울특별시 동대문구 고산자로32길 78, 305-323호 (용두동, 청량리역한양수자...",02-964-1004,02-964-1007
1,2,더드림요양병원,요양병원(일반요양병원),개인,'02580,"서울특별시 동대문구 한빛로 12, 지하1층 ~ 4층 (신설동)",02-6674-7575,02-6674-7576
2,3,엔플러스병원,병원,개인,'02522,"서울특별시 동대문구 장한로32길 7, 장안동의료시설 (장안동)",02-6235-7575,02-6235-7576
3,4,365매일한방병원,한방병원,개인,'02582,"서울특별시 동대문구 천호대로 3, 지하2층, 지상1~2층 (신설동)",02-929-0365,02-928-0365
4,5,엘치과병원,치과병원,개인,'02538,"서울특별시 동대문구 전농로 102, 3층 (답십리동)",02-2246-7528,3394-7510
...,...,...,...,...,...,...,...,...
632,633,고려한의원,한의원,개인,'02473,"서울특별시 동대문구 안암로 130, 2층 (제기동)",02-926-2625,02-928-3560
633,634,박신경정신과의원,의원,개인,'02579,"서울특별시 동대문구 왕산로 45, 3층 301호 (용두동, KY TOWER)",02-923-7610,02-923-7698
634,635,임치과의원,치과의원,개인,'02460,"서울특별시 동대문구 이문로 23-1, 3층 (회기동)",02-967-3496,NaN
635,636,내인당한의원,한의원,개인,'02507,"서울특별시 동대문구 전농로16길 30, 지하1,1층 (전농동)",02-2244-3085,02-2244-4535


In [13]:
def addr_to_admin_dong(addr: str):
    # 1) 주소 → 좌표
    res = api.search_address(addr)
    docs = res.get("documents", [])
    if not docs:
        return None

    doc = docs[0]
    a = doc.get("address") or {}
    x = a.get("x")
    y = a.get("y")
    if not x or not y:
        return None

    # 2) 좌표 → 행정동
    reg = api.geo_coord2regioncode(x, y)
    rdocs = reg.get("documents", [])
    if not rdocs:
        return None

    # 행정동은 region_type == 'H'
    for r in rdocs:
        if r.get("region_type") == "H":
            return r.get("region_3depth_name")

    return None

hos["행정동"] = hos["전처리_도로명주소"].apply(addr_to_admin_dong)

hos.head()
hos.to_excel('행정동변환.xlsx')

In [21]:
#신설동, 용두동, 제기동, 전농1동, 전농2동, 답십리1동, 답십리2동, 장안1동, 장안2동, 청량리동, 회기동, 휘경1동, 휘경2동, 이문1동, 이문2동

#전농1동만 선택
new = hos[hos['행정동']=='전농1동']
new.shape

new.to_excel('전농1동추출.xlsx')

(58, 10)